In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')
# Загрузка данных
data = pd.read_csv('Готовые данные.csv')

# Просмотр структуры данных
print(data.head())
print("\nКоличество записей:", len(data))
print("\nРаспределение по категориям:")
print(data.iloc[:, 3:].sum())

# Подготовка текста
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Удаление спецсимволов и цифр
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text)
    # Приведение к нижнему регистру
    text = text.lower()
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

data['comment'] = data['comment'].apply(preprocess_text)

# Разделение данных
X = data['comment']
y = data.iloc[:, 3:9]  # Категории для классификации

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                        comment       id  rating  \
0                                       спасибо  2945792       5   
1                                      спасибо!  3234340       5   
2                                      Отлично   3380332       5   
3  Благодарю за оперативное решение проблемы !   3381812       5   
4         Прекрасный специалист! Побольше таких  3461991       5   

   Вопрос решен  Нравится качество выполнения заявки  \
0             1                                    0   
1             1                                    0   
2             1                                    1   
3             1                                    1   
4             1                                    0   

   Нравится качество работы сотрудников  Нравится скорость отработки заявок  \
0                                     0                                   0   
1                                     0                                   0   
2                

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np

In [8]:


# Функция для обучения и оценки моделей
def evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test):
    model.fit(X_train_vec, y_train)
    
    # Предсказание вероятностей
    y_pred_proba = model.predict_proba(X_test_vec)
    
    # Вычисление ROC-AUC для каждой категории
    roc_auc_scores = []
    for i in range(len(y_test.columns)):
        roc_auc = roc_auc_score(y_test.iloc[:, i], y_pred_proba[i][:, 1])
        roc_auc_scores.append(roc_auc)
        print(f"ROC-AUC для '{y_test.columns[i]}': {roc_auc:.4f}")
    
    # Средний ROC-AUC
    print(f"\nСредний ROC-AUC: {np.mean(roc_auc_scores):.4f}\n")

# Модели
models = {
    "Logistic Regression": MultiOutputClassifier(LogisticRegression(max_iter=1000)),
    "Random Forest": MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)),
    "Support Vector Machine": MultiOutputClassifier(SVC(kernel='linear', probability=True, random_state=42))
}

# Оценка каждой модели
for model_name, model in models.items():
    print(f"Оценка модели: {model_name}")
    evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test)


Оценка модели: Logistic Regression
ROC-AUC для 'Вопрос решен': 0.9700
ROC-AUC для 'Нравится качество выполнения заявки': 0.8354
ROC-AUC для 'Нравится качество работы сотрудников': 0.9024
ROC-AUC для 'Нравится скорость отработки заявок': 0.9839
ROC-AUC для 'Понравилось выполнение заявки': 0.7142
ROC-AUC для 'Не удовлетворён': 0.9645

Средний ROC-AUC: 0.8951

Оценка модели: Random Forest
ROC-AUC для 'Вопрос решен': 0.8534
ROC-AUC для 'Нравится качество выполнения заявки': 0.8355
ROC-AUC для 'Нравится качество работы сотрудников': 0.9049
ROC-AUC для 'Нравится скорость отработки заявок': 0.9770
ROC-AUC для 'Понравилось выполнение заявки': 0.6545
ROC-AUC для 'Не удовлетворён': 0.8302

Средний ROC-AUC: 0.8426

Оценка модели: Support Vector Machine
ROC-AUC для 'Вопрос решен': 0.9586
ROC-AUC для 'Нравится качество выполнения заявки': 0.7532
ROC-AUC для 'Нравится качество работы сотрудников': 0.9245
ROC-AUC для 'Нравится скорость отработки заявок': 0.9817
ROC-AUC для 'Понравилось выполнение зая